In [1]:
import os 
import time 
import json,math,sys,re
import pandas as pd 

In [ ]:
folder = "../.."
from dotenv import load_dotenv
env_path = f'{folder}/deepseek.env'
load_dotenv(dotenv_path=env_path)
DEEPSEEK_KEY = os.getenv('DEEPSEEK_KEY')
from openai import OpenAI
client = OpenAI(api_key=DEEPSEEK_KEY, base_url="https://api.deepseek.com")

In [69]:
inputfname = "GOLDEN_SET"
inputfname = "GOLDEN_SET_additions"
inputfname = "preE_additions"
inputfname = "cross_refs_missing_CHECK"

# prompt = "P1"
prompt = "P0"
fname = f"{inputfname}_DSV3_{prompt}" 
data = pd.read_csv(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/{inputfname}.csv")
data['index'] = range(0, len(data))
data = data[(data['score'] >= 0.98)]
data = data.to_dict(orient="records")

labels = {}

len(data)

1523

In [ ]:
with open(f"{folder}/QP/predictions/{fname}.json",'r') as f: 
    labels = json.load(f)
len(labels)

In [73]:
from tqdm import tqdm 
start = 0 #
end = len(data) 
my_qp = data[start:end] 

for idx, qp_pair in enumerate(tqdm(my_qp)):
    if idx <= (len(labels) -1 ): continue 
    
    response = client.chat.completions.create(
        model='deepseek-chat',
        messages=[ 
            {"role": "user",  
             "content": f"You are an English and Latin scholar who determines whether the given sentence contains a quotation/paraphrase of a given Biblical verse. You give a single True or False output without any reasoning. \nSENTENCE: '{qp_pair['text']}' \n 'VERSE: {qp_pair['verse_text']}'\n True/False:"}
        ],
        stream=False
    )
    labels[str(qp_pair['index'])] = response.choices[0].message.content
    if (idx+1) % 100 == 0: 
        with open(f"{folder}/QP/predictions/{fname}.json",'w') as f: 
            json.dump(labels,f) 

with open(f"{folder}/QP/predictions/{fname}.json",'w') as f: 
    json.dump(labels,f) 
    # 100%|██████████| 956/956 [2:30:22<00:00,  9.44s/it] 
    # 100%|██████████| 1044/1044 [2:36:31<00:00,  9.00s/it] 
    # 100%|██████████| 4566/4566 [2:51:16<00:00,  2.25s/it]  

  0%|          | 0/1523 [00:00<?, ?it/s]

100%|██████████| 1523/1523 [2:05:25<00:00,  4.94s/it]  


In [91]:
for idx, entry in labels.items(): 
    entry = entry.strip()
    if entry not in ["TRUE","FALSE"]: 
        qp_pair = data[int(idx)]
        print(idx)
        print(qp_pair)
        print(entry)
        # response = client.chat.completions.create(
        #     model="deepseek-chat",
        #     messages=[
        #         {"role": "system", "content": "You are an English and Latin scholar working on Biblical language in early modern texts. You determine whether the first sentence is a quotation/paraphrase of a given Biblical verse in the SAME LANGUAGE. You give a single TRUE or FALSE output FOR EACH USER MESSAGE without any reasoning."},
        #         {"role": "user",  "content": f"SENTENCE: '{qp_pair['text']}'; 'VERSE: {qp_pair['verse_text']}'"}
        #     ],
        #     stream=False
        # )
        # labels[str(qp_pair['index'])] = response.choices[0].message.content
        # print(response.choices[0].message.content)

In [86]:
with open(f"../../DH/Early-Modern-Sermons/assets/QP_labels/{fname}.json",'w') as f: 
    json.dump(labels,f)

len(labels)

3001